In [1]:
#Import packages needed
import pandas as pd
import numpy as np
import gmaps
import requests
import re
from config import gkey
from pprint import pprint
gmaps.configure(api_key=gkey)
import matplotlib.pyplot as plt

In [49]:
# Bring in csv and look at data
# jobscat_csv = "../../Archive/completeAndCleanNycJobs_kh.csv"
jobscat_df = pd.read_csv("../../Archive/completeAndCleanNycJobs_kh.csv", dtype={'zip': 'str'})
jobscat_df.head()

,index,jobID,agency,postingType,numOfPositions,businessTitle,civilServiceTitle,titleCodeNo,level,jobCategory,...,residencyRequirement,postingDate,postingUpdated,processDate,newLat,newLong,formattedAddress,zip,borough,jobCatCondensed
0,23,192723,DEPT OF INFO TECH & TELECOMM,Internal,1,Research Assistant,RESEARCH ASSISTANT,60910,0,Clerical & Administrative Support Legal,...,New York City residency is generally required ...,2015-05-18T00:00:00.000,2015-05-18T00:00:00.000,2019-07-30T00:00:00.000,40.714463,-74.010980,"75 Park Pl, New York, NY 10007, USA",10007.0,Manhattan,"Technical, Sales, and Administrative"
1,45,226045,DEPT OF INFO TECH & TELECOMM,Internal,1,Senior Civil Service Advisor,ASSOCIATE STAFF ANALYST,12627,0,Clerical & Administrative Support Policy & Ana...,...,New York City residency is generally required ...,2015-12-17T00:00:00.000,2015-12-17T00:00:00.000,2019-07-30T00:00:00.000,40.714463,-74.010980,"75 Park Pl, New York, NY 10007, USA",10007.0,Manhattan,"Technical, Sales, and Administrative"
2,88,242604,POLICE DEPARTMENT,Internal,3,Civilian Investigator,INVESTIGATOR,31105,0,"Public Safety, Inspections, & Enforcement",...,New York City residency is generally required ...,2016-06-07T00:00:00.000,2016-07-19T00:00:00.000,2019-07-30T00:00:00.000,40.712110,-74.001886,"1 Police Plaza Path, New York, NY 10007, USA",10007.0,Manhattan,Managerial and Professional
3,89,242877,LAW DEPARTMENT,Internal,3,Claim Specialist,CLAIM SPECIALIST,30726,1,Clerical & Administrative Support Legal,...,New York City residency is generally required ...,2016-06-15T00:00:00.000,2016-07-21T00:00:00.000,2019-07-30T00:00:00.000,40.703323,-74.008896,"55 Water Street, 55 Water St, New York, NY 100...",10041.0,Manhattan,"Technical, Sales, and Administrative"
4,90,242877,LAW DEPARTMENT,Internal,3,Claim Specialist,CLAIM SPECIALIST,30726,1,Clerical & Administrative Support Legal,...,New York City residency is generally required ...,2016-06-15T00:00:00.000,2016-07-21T00:00:00.000,2019-07-30T00:00:00.000,40.703323,-74.008896,"55 Water Street, 55 Water St, New York, NY 100...",10041.0,Manhattan,"Technical, Sales, and Administrative"


In [69]:
boroughgroup = jobscat_df.groupby(["borough", "jobCatCondensed","agency","zip"])\
                                    .agg({'zip': "count", 
                                          "numOfPositions": "sum",
                                         'salaryRangeFrom': "min",
                                         'salaryRangeTo':"max"})
boroughgroup["salaryRangeFrom"] = boroughgroup["salaryRangeFrom"].map("${:,.2f}".format)
boroughgroup["salaryRangeTo"] = boroughgroup["salaryRangeTo"].map("${:,.2f}".format)

boroughgroup = boroughgroup.rename(columns={"borough": "Borough",
                                            "zip":"Jobs in Zip Code",
                                    "numOfPositions":"Number of Positions",
                                    "salaryRangeFrom": "Salary Minimun",
                                    "salaryRangeTo": "Salary Max"})
# df1.rename_axis(['names', None, 'Colors'])
boroughgroup = boroughgroup.rename_axis(["Borough", "Job Category","Department/Agency", "Zip Code"])
boroughgroup

Jobs in Zip Code  \
Borough       Job Category                         Department/Agency              Zip Code                     
Brooklyn      Managerial and Professional          DEPARTMENT OF BUILDINGS        11222.0                  6   
                                                   DEPARTMENT OF TRANSPORTATION   11222.0                  2   
              Operatives and Laborers              DEPARTMENT OF TRANSPORTATION   11222.0                  2   
                                                   DEPT OF ENVIRONMENT PROTECTION 11222.0                  2   
                                                   DEPT. OF HOMELESS SERVICES     11236.0                  4   
              Other                                DEPT OF ENVIRONMENT PROTECTION 10595.0                  2   
                                                                                  11222.0                  1   
                                                   DEPT OF INFO TECH & TELECOMM   11201.0                  4   
                                                   OFFICE OF EMERGENCY MANAGEMENT 11201.0                 12   
                                                   POLICE DEPARTMENT              11216.0                  1   
              Service                              DEPARTMENT OF BUILDINGS        11222.0                  1   
                                                   HRA/DEPT OF SOCIAL SERVICES    11201.0                  1   
                                                   LAW DEPARTMENT                 11201.0                  2   
Manhattan     Managerial and Professional          BOARD OF CORRECTION            10007.0                  2   
                                                   DEPARTMENT OF TRANSPORTATION   10041.0                 16   
                                                   DEPT. OF HOMELESS SERVICES     10004.0                  3   
                                                   HRA/DEPT OF SOCIAL SERVICES    10007.0                  2   
                                                   POLICE DEPARTMENT              10007.0                  1   
              Operatives and Laborers              DEPT OF ENVIRONMENT PROTECTION 10595.0                  2   
                                                   DEPT OF RECORDS & INFO SERVICE 10007.0                  2   
                                                   HRA/DEPT OF SOCIAL SERVICES    10007.0                  1   
              Other                                BOARD OF CORRECTION            10007.0                  4   
                                                   DEPARTMENT OF FINANCE          10007.0                  3   
                                                                                  10038.0                  5   
                                                   DEPARTMENT OF SANITATION       10004.0                  6   
                                                                                  10013.0                  4   
                                                   DEPARTMENT OF TRANSPORTATION   10041.0                 43   
                                                   DEPT OF ENVIRONMENT PROTECTION 10007.0                 10   
                                                                                  10038.0                  4   
                                                                                  12401.0                  2   
                                                                                  12740.0                  2   
                                                   DEPT OF RECORDS & INFO SERVICE 10007.0                  2   
                                                   LAW DEPARTMENT                 10007.0                  4   
                                                   POLICE DEPARTMENT              10007.0                 11   
              Service                              DEPT OF Y

In [85]:
BrooklynGroup = (boroughgroup[boroughgroup.index.get_level_values("Borough").str.startswith('Brook')])
ManhattanGroup = (boroughgroup[boroughgroup.index.get_level_values("Borough").str.startswith('Man')])
QueensGroup = (boroughgroup[boroughgroup.index.get_level_values("Borough").str.startswith('Queens')])
StatenIslandGroup = (boroughgroup[boroughgroup.index.get_level_values("Borough").str.startswith('Staten')])
BronxGroup = (boroughgroup[boroughgroup.index.get_level_values("Borough").str.startswith('The')])

In [86]:
BrooklynGroup.to_html("Brooklyn.html")